# Simple illustration of DMRG for spin-1/2 Heisenberg model

This notebook follows the paper

    Eric Jeckelmann, Density-Matrix Renormalization Group Algorithms, 2008
    
This paper nicely introduces the numerical renormalization group (NRG), infinite-system DMRG, and the finite system DMRG in a concise way for the spin-1/2 Heisenberg model.

$$H=\sum_{i=1}^{N-1} \mathbf{S}_{i}\cdot\mathbf{S}_{i+1}.$$

This is a modified version of the Julia code on the github `simple-dmrg` by James R. Garrison and Ryan V. Mishmash 

https://simple-dmrg.readthedocs.io/en/latest/

The implementation below is compatible with Julia 1.0.

This codes treats the left and right blocks of DMRG separately.

<br>

Lin Lin

Last revision: 9/21/2018
<br><br>



Initial setup of the Hamiltonian. The "connecting" part of the Hamiltonian is given by

$\mathbf{S}_{i}\cdot\mathbf{S}_{i+1} = S^{z}_{i} S^{z}_{i+1} + \frac12 (S^{+}_i S^{-}_{i+1}+S^{-}_i S^{+}_{i+1})$
 
`enlarge_block` increases the system by one site from $j$ to $j+1$. Hence the tensor product of the the previous Hamiltonian with the identity is needed, plus the new connecting part of the Hamiltonian.  

The new connecting component $S_{j}^z$ and $S_{j}^{+}$ is also defined on the enlarged space via the tensor product. Note that `kron` uses the tensor product convention making that the second block is indexed first. In other words, Julia uses the column-major convention for vectors and matrices, but the ordering of the blocks follows the row-major convention.

The infinite DMRG algorithm (at least the infinite version) contains the following steps. 

1. Construct the superblock Hamiltonian via the tensor product of the previously compressed components.  This is not a memory-efficient implementation since it does not use the low-rank structure of the superblock Hamiltonian.

2. The update of the MPS basis is always on the left, by performing an eigenvalue decomposition of the reduced density matrix on the new site on the left. The update on the right block mirrors the update on the left.


The finite DMRG algorithm first performs a "warmup" sweeping step using the infinite DMRG algorithm to reach site L. Then the blocks are stored on a "disk" (in this case memory).  In the consequent sweeping steps, the finite system DMRG first sweeps from left to right, and then right to left. The use of the name `sys` and `env` rather than `left` and `right` is mainly for the finite system DMRG, where `sys` and `env` alternatively assumes the role of `left` and `right` in different sweeping steps. 

Unlike the infinite-system algorithm, the `env` part always reads the MPS basis from the "disk" (i.e. the valuesfrom the previous sweep), and correspondingly the "disk" is updated by the `sys` part.


In [2]:
using Arpack
using LinearAlgebra
using SparseArrays

# The length of the EnlargedBlock is always the length of the
# corresponding Block plus 1
struct Block
    length::Int
    basis_size::Int
    operator_dict::Dict{Symbol,AbstractMatrix{Float64}}
end

struct EnlargedBlock
    length::Int
    basis_size::Int
    operator_dict::Dict{Symbol,AbstractMatrix{Float64}}
end

# Model-specific code for the Heisenberg XXZ chain
model_d = 2  # single-site basis size

Sz1 = [0.5 0.0; 0.0 -0.5]  # single-site S^z
Sp1 = [0.0 1.0; 0.0 0.0]  # single-site S^+

H1 = [0.0 0.0; 0.0 0.0]  # single-site portion of H is zero

J = 1.0 
Jz = 1.0 

function H2(Sz1, Sp1, Sz2, Sp2)  # two-site part of H
    # Given the operators S^z and S^+ on two sites in different Hilbert spaces
    # (e.g. two blocks), returns a Kronecker product representing the
    # corresponding two-site term in the Hamiltonian that joins the two sites.
    return (J / 2) * (kron(Sp1, Sp2') + kron(Sp1', Sp2)) + Jz * kron(Sz1, Sz2)
end

# conn refers to the connection operator, that is, the operator on the edge of
# the block, on the interior of the chain. 
#
# In Block, conn_* operators are represented in the current MPS basis.
#
# In EnlargedBlock, conn_* are simply the tensor product of * operators
# with proper identity tensors.
initial_block = Block(1, model_d, 
    Dict{Symbol,AbstractMatrix{Float64}}(
    :H => H1,
    :conn_Sz => Sz1,
    :conn_Sp => Sp1)
)

function enlarge_block(block::Block, block_label::Symbol)
    # This function enlarges the provided Block by a single site, returning an
    # EnlargedBlock.
    #
    # This way how tensor product is performed depends on whether the
    # block is on the left or on the right, following a consistent
    # convention of Kronecker products.
    mblock = block.basis_size
    o = block.operator_dict

    # Create the new operators for the enlarged block.  
    I1 = sparse(1.0I, model_d, model_d)
    I_block = sparse(1.0I, mblock, mblock)
    if( block_label == :l )
        enlarged_operator_dict = Dict{Symbol,AbstractMatrix{Float64}}(
            :H => kron(o[:H], I1) + kron(I_block, H1) + H2(o[:conn_Sz], o[:conn_Sp], Sz1, Sp1),
            :conn_Sz => kron(I_block, Sz1),
            :conn_Sp => kron(I_block, Sp1))
    elseif( block_label == :r )
        enlarged_operator_dict = Dict{Symbol,AbstractMatrix{Float64}}(
            :H => kron(I1, o[:H]) + kron(H1, I_block) + H2(Sz1, Sp1, o[:conn_Sz], o[:conn_Sp]),
            :conn_Sz => kron(Sz1, I_block),
            :conn_Sp => kron(Sp1, I_block))
    else
        error("Wrong label for enlarge_block.")
    end
    
    return EnlargedBlock(block.length + 1,
                         block.basis_size * model_d,
                         enlarged_operator_dict)
end


function solve_super_hamiltonian(blocks_enl::Dict{Symbol,EnlargedBlock})
    # Construct the full superblock Hamiltonian corresponding to the two
    # sites in the middle.
    m_blockL_enl = blocks_enl[:l].basis_size
    m_blockR_enl = blocks_enl[:r].basis_size
    I_blockL_enl = sparse(1.0I, m_blockL_enl, m_blockL_enl)
    I_blockR_enl = sparse(1.0I, m_blockR_enl, m_blockR_enl)

    H_L       = blocks_enl[:l].operator_dict[:H]
    conn_Sz_L = blocks_enl[:l].operator_dict[:conn_Sz]
    conn_Sp_L = blocks_enl[:l].operator_dict[:conn_Sp]

    H_R       = blocks_enl[:r].operator_dict[:H]
    conn_Sz_R = blocks_enl[:r].operator_dict[:conn_Sz]
    conn_Sp_R = blocks_enl[:r].operator_dict[:conn_Sp]

    superblock_hamiltonian = kron(H_L, I_blockR_enl) +
        kron(I_blockL_enl, H_R) +
        H2(conn_Sz_L, conn_Sp_L, conn_Sz_R, conn_Sp_R) 
    superblock_hamiltonian = (superblock_hamiltonian +
                              superblock_hamiltonian') / 2.0

    # Call ARPACK to find the superblock ground state.  (:SR means find the
    # eigenvalue with the "smallest real" value.)
    (energy,), psi0 = eigs(superblock_hamiltonian, nev=1, which=:SR)
    return (energy, psi0)
end


function update_block(blocks_enl::Dict{Symbol,EnlargedBlock}, psi0,
                      m::Int, sys_label::Symbol)
    # Construct the reduced density matrix of the system by tracing out
    # the environment. 
    #
    # (sys_label, env_label) should be (:l,:r) or (:r,:l), respectively.
    #

    m_blockL_enl = blocks_enl[:l].basis_size
    m_blockR_enl = blocks_enl[:r].basis_size


    # rotate the system part
    
    # We want to make the (sys, env) indices correspond to (row, column) of a
    # matrix, respectively.  Since the environment (column) index updates most
    # quickly in our Kronecker product structure, psi0 is thus row-major.
    # However, Julia stores matrices in column-major format, so we first
    # construct our matrix in (env, sys) form and then take the transpose.
    psi0_t = transpose(reshape(psi0, (m_blockR_enl, m_blockL_enl)))

    fact = svd(psi0_t)
    evals = fact.S.^2
    if( sys_label == :l )
        evecs = fact.U
    else
        evecs = fact.V
    end

    # Build the transformation matrix from the `m` overall most significant
    # eigenvectors.
    my_m = min(length(evals), m)
    indices = 1:my_m
    basis = evecs[:, indices]

    truncation_error = 1 - sum(evals[indices])
    println("truncation error: ", truncation_error)

    # Rotate and truncate each operator.
    new_operator_dict = Dict{Symbol,AbstractMatrix{Float64}}()
    for (name, op) in blocks_enl[sys_label].operator_dict
        new_operator_dict[name] = basis' * (op * basis)
    end

    newblock = Block(blocks_enl[sys_label].length, my_m, new_operator_dict)
    return newblock
end


function infinite_dmrg_step(blocks::Dict{Symbol,Block}, m::Int)
    # Performs a single infinite DMRG step, and keeping a maximum of `m`
    # states in the new basis.
    #
    # Infinite DMRG algorithm uses the same psi0 to update the system
    # and environment part.


    # Enlarge each block by a single site.
    blocks_enl = Dict{Symbol,EnlargedBlock}()
    
    blocks_enl[:l] = enlarge_block(blocks[:l],:l)
    blocks_enl[:r] = enlarge_block(blocks[:r],:r)

    (energy, psi0) = solve_super_hamiltonian( blocks_enl )
    
    newblocks = Dict{Symbol,Block}()
    newblocks[:l] = update_block( blocks_enl, psi0, m, :l )
    newblocks[:r] = update_block( blocks_enl, psi0, m, :r )

    return newblocks, energy
end

function finite_dmrg_step(blocks::Dict{Symbol,Block}, m::Int,
                          sys_label::Symbol)
    # Performs a single finite DMRG step, and keeping a maximum of `m`
    # states in the new basis.
    #
    # Finite DMRG algorithm only updates the system part, and keeps the
    # environment part intact.

    # Enlarge each block by a single site.
    blocks_enl = Dict{Symbol,EnlargedBlock}()
    
    blocks_enl[:l] = enlarge_block(blocks[:l],:l)
    blocks_enl[:r] = enlarge_block(blocks[:r],:r)

    (energy, psi0) = solve_super_hamiltonian( blocks_enl )

    # Only update the system part
    newblocks = copy( blocks )
    newblocks[sys_label] = update_block( blocks_enl, psi0, m, sys_label )

    return newblocks, energy
end



function graphic(blocks::Dict{Symbol,Block}, sys_label::Symbol)
    # Returns a graphical representation of the DMRG step we are about to
    # perform, using '=' to represent the system sites, '-' to represent the
    # environment sites, and '**' to represent the two intermediate sites.

    if( sys_label == :l )
        str = repeat("=", blocks[:l].length) * "**" * repeat("-", blocks[:r].length)
    elseif( sys_label == :r )
        str = repeat("-", blocks[:l].length) * "**" * repeat("=", blocks[:r].length)
    else
        error("Wrong label.")
    end

    return str
end



function infinite_system_algorithm(L::Int, m::Int)
    # Repeatedly enlarge the system by performing a single DMRG step, using a
    # reflection of the current block as the environment.
    blocks = Dict{Symbol,Block}()
    blocks[:l] = initial_block
    blocks[:r] = initial_block
    println("Infinite-system DMRG sweeping to with bond dimension m = ", m)

    while blocks[:l].length + blocks[:r].length < L
        println(graphic(blocks, :l))
        current_L = blocks[:l].length + blocks[:r].length + 2
        println("L = ", current_L)
        blocks, energy = infinite_dmrg_step(blocks, m)
        println("E/L = ", energy / current_L )
    end
end


function finite_system_algorithm(L::Int, m_warmup::Int, m_sweep_list::AbstractVector{Int})
    @assert iseven(L)

    # To keep things simple, this dictionary is not actually saved to disk, but
    # we use it to represent persistent storage.
    block_disk = Dict{Tuple{Symbol,Int},Block}()  # "disk" storage for Block objects

    # Use the infinite system algorithm to build up to desired size.  Each time
    # we construct a block, we save it for future reference as both a left
    # (:l) and right (:r) block, as the infinite system algorithm assumes the
    # environment is a mirror image of the system.
    blocks = Dict{Symbol,Block}()
    blocks[:l] = initial_block
    blocks[:r] = initial_block
    
    block_disk[:l, blocks[:l].length] = blocks[:l]
    block_disk[:r, blocks[:r].length] = blocks[:r]
    println("Infinite-system DMRG sweeping to warm up with bond dimension m = ", m_warmup)
    while blocks[:l].length + blocks[:r].length < L
        # Perform a single DMRG step and save the new Block to "disk"
        println(graphic(blocks, :l))
        current_L = blocks[:l].length + blocks[:r].length + 2
        blocks, energy = infinite_dmrg_step(blocks, m_warmup)
        println("E/L = ", energy / current_L )
        block_disk[:l, blocks[:l].length] = blocks[:l]
        block_disk[:r, blocks[:r].length] = blocks[:r]
    end

    # Now that the system is built up to its full size, we perform sweeps using
    # the finite system algorithm.  At first the left block will act as the
    # system, growing at the expense of the right block (the environment), but
    # once we come to the end of the chain these roles will be reversed.
    sys_label, env_label = :l, :r

    for m in m_sweep_list
        println()
        println("Finite-system DMRG with bond dimension m = ", m)
        while true
            # Load the appropriate environment block from "disk"
            blocks[env_label] = block_disk[env_label, L - blocks[sys_label].length - 2]
            if blocks[env_label].length == 1
                # We've come to the end of the chain, so we reverse course.
                sys_label, env_label = env_label, sys_label
            end

            # Perform a single DMRG step.
            println(graphic(blocks, sys_label))
            blocks, energy = finite_dmrg_step(blocks, m, sys_label)

            println("E/L = ", energy / L)
            println("E   = ", energy)

            # Save the block from this step to disk.
            block_disk[sys_label, blocks[sys_label].length] = blocks[sys_label]

            # Check whether we just completed a full sweep.
            if sys_label == :l && (2 * blocks[sys_label].length == L)
                break  # escape from the "while true" loop
            end
        end
    end
end


finite_system_algorithm (generic function with 1 method)

In the following, the graphical form `====**----` indicates the progress of the DMRG algorithms. Here `=` means the system sites, `-` means the environment sites, and `**` means the two intermediate sites.

In [3]:
println("Infinite system algorithm.")
infinite_system_algorithm(20, 4)


Infinite system algorithm.
Infinite-system DMRG sweeping to with bond dimension m = 4
=**-
L = 4
truncation error: 4.440892098500626e-16
truncation error: 4.440892098500626e-16
E/L = -0.40400635094610965
==**--
L = 6
truncation error: 0.00035047202327176397
truncation error: 0.00035047202327176397
E/L = -0.41559618898132095
===**---
L = 8
truncation error: 2.063959231901613e-5
truncation error: 2.063959231901613e-5
E/L = -0.42148189080112886
====**----
L = 10
truncation error: 0.0008173920482934527
truncation error: 0.0008173920482934527
E/L = -0.4254039796671555
=====**-----
L = 12
truncation error: 5.2738121927520254e-5
truncation error: 5.2738121927520254e-5
E/L = -0.4277067411276085
======**------
L = 14
truncation error: 0.0010624733433289846
truncation error: 0.0010624733433289846
E/L = -0.4296795574121556
=======**-------
L = 16
truncation error: 6.955269682840104e-5
truncation error: 6.955269682840104e-5
E/L = -0.43085100873627785
========**--------
L = 18
truncation error: 0.0

In [4]:

println("Finite system algorithm.")
finite_system_algorithm(20, 4, [4, 4])


Finite system algorithm.
Infinite-system DMRG sweeping to warm up with bond dimension m = 4
=**-
truncation error: 0.0
truncation error: 0.0
E/L = -0.40400635094610937
==**--
truncation error: 0.00035047202327120885
truncation error: 0.00035047202327120885
E/L = -0.41559618898132106
===**---
truncation error: 2.0639592318127953e-5
truncation error: 2.0639592318127953e-5
E/L = -0.4214818908011289
====**----
truncation error: 0.0008173920482944519
truncation error: 0.0008173920482944519
E/L = -0.4254039796671555
=====**-----
truncation error: 5.273812192685412e-5
truncation error: 5.273812192685412e-5
E/L = -0.4277067411276083
======**------
truncation error: 0.0010624733433302058
truncation error: 0.0010624733433302058
E/L = -0.42967955741215574
=======**-------
truncation error: 6.955269682784593e-5
truncation error: 6.955269682784593e-5
E/L = -0.4308510087362784
========**--------
truncation error: 0.0011659043406294645
truncation error: 0.0011659043406294645
E/L = -0.4320486002127136